In [1]:
! pip install haystack-ai==2.19
! pip install colorama
! pip install ollama-haystack
! pip install "sentence-transformers>=4.1.0"

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from typing import List
from pydantic import BaseModel
from datetime import date
import json
import random
import pydantic
from pydantic import ValidationError
from typing import Optional, List
from colorama import Fore
from haystack import component
from haystack.dataclasses import ChatMessage

<font size="6">Exercise 1b defing datastructure for the extraction of tasks</font>


In [ ]:
import json 

class Task(BaseModel):
    name: str

class TasksData(BaseModel):
    tasks: List[Task]

json_schema = TasksData.model_json_schema()

In [4]:
import json

json_schema = TasksData.model_json_schema()

In [6]:
# Define the component input parameters
@component
class OutputValidator:
    def __init__(self, pydantic_model: pydantic.BaseModel):
        self.pydantic_model = pydantic_model
        self.iteration_counter = 0

    # Define the component output
    @component.output_types(valid_replies=List[ChatMessage], invalid_replies=List[ChatMessage], error_message=Optional[str])
    def run(self, replies: List[ChatMessage]):

        self.iteration_counter += 1

        ## Try to parse the LLM's reply ##
        # If the LLM's reply is a valid object, return `"valid_replies"`
        try:
            output_dict = json.loads(replies[0].text)
            self.pydantic_model.model_validate(output_dict)
            print(
                Fore.GREEN
                + f"OutputValidator at Iteration {self.iteration_counter}: Valid JSON from LLM - No need for looping: {replies[0]}"
            )
            return {"valid_replies": replies}

        # If the LLM's reply is corrupted or not valid, return "invalid_replies" and the "error_message" for LLM to try again
        except (ValueError, ValidationError) as e:
            print(
                Fore.RED
                + f"OutputValidator at Iteration {self.iteration_counter}: Invalid JSON from LLM - Let's try again.\n"
                f"Output from LLM:\n {replies[0]} \n"
                f"Error from OutputValidator: {e}"
            )
            return {"invalid_replies": replies, "error_message": str(e)}


 <font size="6">Exercise 1c The component “OutputValidator” by using our datastructure-template as a variable</font>


In [7]:
validator = OutputValidator(pydantic_model=TasksData)

 <font size="6">Exercise 1d prompt to instruct an LLM to extract task names from a BPMN description</font>


In [ ]:
from haystack.components.builders import ChatPromptBuilder

prompt_template = [
    ChatMessage.from_user(
        """
Extract all *task names* from the following BPMN description:

{{bpmn_description}}

Return ONLY JSON in the following format:

{
  "tasks": [
    {"name": "task name 1"},
    {"name": "task name 2"}
  ]
}

RULES:
- Extract only task NAMES.
- Output must be valid JSON.
- Do NOT include explanations.
- Do NOT include code fences.
- Do NOT include the schema.

{% if invalid_replies and error_message %}
Your previous output was invalid:

{{invalid_replies}}

Error:

{{error_message}}

Please correct your answer and return valid JSON.
{% endif %}
"""
    )
]

required_vars = ["bpmn_description"]

prompt_builder = ChatPromptBuilder(
    template=prompt_template,
    required_variables=required_vars
)


 <font size="6">Excersie 1e generator with the OllamaChatGenerator Llama3.1</font>


In [ ]:
import os
from getpass import getpass

from haystack_integrations.components.generators.ollama import  OllamaChatGenerator

chat_generator = OllamaChatGenerator(
    model="llama3.1:8b",
    url="http://localhost:11434",
    timeout=30*60,  
    generation_kwargs={
        "num_ctx": 4096,
        "temperature": 0.9,
    }
)

In [10]:
from haystack import Pipeline

pipeline = Pipeline(max_runs_per_component=5)

pipeline.add_component("prompt_builder", prompt_builder)
pipeline.add_component("llm", chat_generator)
pipeline.add_component("validator", validator)

pipeline.connect("prompt_builder.prompt", "llm.messages")
pipeline.connect("llm.replies", "validator.replies")




🚅 Components
  - prompt_builder: ChatPromptBuilder
  - llm: OllamaChatGenerator
  - validator: OutputValidator
🛤️ Connections
  - prompt_builder.prompt -> llm.messages (list[ChatMessage])
  - llm.replies -> validator.replies (List[ChatMessage])

 <font size="6">Excersie 1f Running the pipeline for 3 different descriptive texts</font>


In [ ]:

bpmn_inputs = [
    """
    The process starts when a customer submits an order.
    Then the system validates the order.
    Next, the warehouse picks the items.
    After that, the items are packed and shipped.
    Finally, the customer receives the package.
    """,
    """
    A user logs into the platform.
    The system checks credentials.
    If valid, a dashboard is displayed.
    The user can then upload files.
    The process ends when the user logs out.
    """,
    """
    A visitor fills out a contact form.
    The system stores the request.
    An agent reviews the request.
    The agent sends a response email.
    The ticket is then closed.
    """
]


for i, bpmn_description in enumerate(bpmn_inputs, start=1):
    print(f"\n====================== BPMN RUN {i} ======================\n")

    result = pipeline.run({
        "prompt_builder": {
            "bpmn_description": bpmn_description
        }
    })

    # Extract valid reply
    if "validator" in result and result["validator"].get("valid_replies"):
        valid_output = result["validator"]["valid_replies"][0].text
        print("✅ Valid JSON Output:\n", valid_output)

        parsed = TasksData.model_validate_json(valid_output)
        print("\nParsed tasks:", [t.name for t in parsed.tasks])

    else:
        print("❌ No valid reply found for this BPMN.")


====================== BPMN RUN 1 ======================

OutputValidator at Iteration 1: Valid JSON from LLM - No need for looping: ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='{\n  "tasks": [\n    {"name": "Validate order"},\n    {"name": "Pick items from warehouse"},\n    {"name": "Pack and ship items"},\n    {"name": "Deliver package to customer"}\n  ]\n}')], _name=None, _meta={'model': 'llama3.1:8b', 'done': True, 'total_duration': 16527908878, 'load_duration': 1201863360, 'prompt_eval_duration': 7082000000, 'eval_duration': 8242000000, 'logprobs': None, 'finish_reason': 'stop', 'completion_start_time': '2025-12-02T21:49:20.494050847Z', 'usage': {'completion_tokens': 49, 'prompt_tokens': 145, 'total_tokens': 194}})
✅ Valid JSON Output:
 {
  "tasks": [
    {"name": "Validate order"},
    {"name": "Pick items from warehouse"},
    {"name": "Pack and ship items"},
    {"name": "Deliver package to customer"}
  ]
}

Parsed tasks: ['Validate order', 

Order Fulfillment Process
The model was able to identify 4 tasks out of 5 
Precision = 1 Recall = 0.8
User Login
The model was able to identify 5 tasks out of 5
Precision = 1 Recall = 1
Customer support
The model was able to identify 5 tasks out of 5
Precision = 1 Recall = 1

<font size="6">Exercise 2a(1) Chat PromptBuilder that receives a query containing a BPMN model and its textual description</font>


In [ ]:
from haystack.components.builders import ChatPromptBuilder

second_prompt_template = ChatPromptBuilder(
    template=[
        ChatMessage.from_user(
            """
You are given a BPMN model and its textual description.
Your task is to annotate the text by wrapping each BPMN element with its corresponding tag.
For example:
  - A task should be wrapped as: <bpmn:task>Task Name</bpmn:task>
  - A start event as: <bpmn:startEvent>Event Description</bpmn:startEvent>
  - A gateway as: <bpmn:exclusiveGateway>Decision Point</bpmn:exclusiveGateway>

Here is the BPMN model: {{bpmn_model}}
Here is the text: {{text_description}}
Annotate the text below:

"""
        )
    ],
    required_variables=["bpmn_model","text_description"] 
)

<font size="6">Exercise 2a(2) A generator that works with the BPMN model</font>

In [ ]:
from haystack_integrations.components.generators.ollama import  OllamaChatGenerator

second_chat_generator = OllamaChatGenerator(
    model="llama3.1:8b",
    url="http://localhost:11434",
    timeout=30 * 60,
    generation_kwargs={
        "num_ctx": 4096,
        "temperature": 0.7, 
    }
)

In [7]:
from haystack import Pipeline

second_pipeline = Pipeline()
second_pipeline.add_component("prompt", second_prompt_template)
second_pipeline.add_component("llm", second_chat_generator)
second_pipeline.connect("prompt.prompt", "llm.messages")



🚅 Components
  - prompt: ChatPromptBuilder
  - llm: OllamaChatGenerator
🛤️ Connections
  - prompt.prompt -> llm.messages (list[ChatMessage])

<font size="6">Exercise 2b Running pipeline for 3 different BPMN-text pairs</font>

In [ ]:
TEXT_FILE = "Passenger Security.txt"
BPMN_FILE = "Passanger_security.bpmn"  

with open(TEXT_FILE, "r", encoding="utf-8") as f:
    text_description = f.read().strip()
with open(BPMN_FILE, "r", encoding="utf-8") as f:
    bpmn_model = f.read().strip()

result = second_pipeline.run({
    "prompt": {
        "text_description": text_description,
        "bpmn_model": bpmn_model
    }
})

annotated = result["llm"]["replies"][0].text.strip()
print(Fore.GREEN + "\n✅ Annotated Output:\n")
print(annotated)


✅ Annotated Output:

Here is the annotated text:

<bpmn:startEvent>Passenger presents their boarding pass</bpmn:startEvent>
at the <bpmn:startEvent>Start Event</bpmn:startEvent>.
After this, the passenger proceeds to the <bpmn:task>Security Check</bpmn:task>,
where security personnel conduct the initial screening.
Once the check is completed, an <bpmn:exclusiveGateway>Exclusive Gateway</bpmn:exclusiveGateway>
evaluates whether the passenger appears suspicious.

If the passenger is not suspicious, the flow continues normally to the <bpmn:task>Manual Control</bpmn:task>,
where additional checks are carried out. When manual control is finished,
the process loops back to the <bpmn:exclusiveGateway>Exclusive Gateway</bpmn:exclusiveGateway>
to determine whether the passenger's destination lies within the Schengen area.

If the destination is within the Schengen zone, the passenger moves on to <bpmn:task>Passport Control</bpmn:task>,
where immigration officers verify their passport. 
If the 

Passenger Security
The model were able to annotate 0 events out of 2, 4 tasks out of 7 and 2 gateways out of 4
Precision =0.85 Recall = 0.428

In [ ]:
TEXT_FILE_2 = "Excese_bagage.txt"
BPMN_FILE_2 = "Excese_bagage.bpmn"  

with open(TEXT_FILE_2, "r", encoding="utf-8") as f:
    text_description = f.read().strip()

with open(BPMN_FILE_2, "r", encoding="utf-8") as f:
    bpmn_model = f.read().strip()

result = second_pipeline.run({
    "prompt": {
        "text_description": text_description,
        "bpmn_model": bpmn_model
    }
})

annotated = result["llm"]["replies"][0].text.strip()
print(Fore.GREEN + "\n✅ Annotated Output:\n")
print(annotated)


✅ Annotated Output:

Here is the annotated text:

Managing Excess Carry-On Baggage Due to High Occupancy 

The <bpmn:process> begins when the passenger checks in for the flight. After check-in, the <bpmn:task> "Assess Flight Occupancy" evaluates the flight’s occupancy level.
An <bpmn:exclusiveGateway> then determines whether the occupancy is high. If it is high, the process continues with the team enhancing baggage control by executing the <bpmn:task> "Enhance baggage control", 
after which passengers are notified about strict carry-on enforcement through the execution of the <bpmn:task> "Notify passengers of strict carry-on enforcement". If the occupancy is not high, the team instead applies the standard carry-on procedures by executing the <bpmn:task> "apply standard carry-on procedures".
Following this, the gate staff receive a notification regarding the flight situation through the execution of the <bpmn:task> "Receive notification", and the passenger eventually arrives at the gat

Execs Baggage 
The model were able to annotate 1 events out of 2,  8 tasks out of 12 and 2 gateways out of 3
Precision = 0.85  Recall = 0.61

In [ ]:
TEXT_FILE_3 = "Delayed Baggage.txt"
BPMN_FILE_3 = "Delayed_bagage.bpmn"  

with open(TEXT_FILE_3, "r", encoding="utf-8") as f:
    text_description = f.read().strip()

with open(BPMN_FILE_3, "r", encoding="utf-8") as f:
    bpmn_model = f.read().strip()

result = second_pipeline.run({
    "prompt": {
        "text_description": text_description,
        "bpmn_model": bpmn_model
    }
})

annotated = result["llm"]["replies"][0].text.strip()
print(Fore.GREEN + "\n✅ Annotated Output:\n")
print(annotated)


✅ Annotated Output:

Here is the annotated text:

The process begins with the start event <bpmn:startEvent>Passenger Arrives at Destination Without Baggage</bpmn:startEvent>, 
after which the passenger performs the task <bpmn:task>Report Delayed Baggage</bpmn:task>. The Baggage Service Office then executes <bpmn:task>Receive report</bpmn:task> followed by <bpmn:task>Initiate Baggage Tracing</bpmn:task>. 
The Baggage Logistics System carries out <bpmn:task>traces baggage across airlines</bpmn:task>, 
after which the process reaches the exclusive gateway <bpmn:exclusiveGateway>Is Baggage Located Within 24 Hours?</bpmn:exclusiveGateway>&mdash;if yes, 
it proceeds to <bpmn:task>Proceed to delivery</bpmn:task>; if no, it performs <bpmn:task>Continue tracing</bpmn:task> and loops back to reassess. 
Once the baggage is ready for delivery, the Baggage Service Office performs <bpmn:task>activate interim support</bpmn:task>, 
which is followed by <bpmn:task>Receive interim support</bpmn:task>. 

Delayed Baggage
The model were able to annotate 0 events out of 2 , 13 tasks out of 13  and 2 gateways out of 4 
Precision = 1 Recall = 0.79

In [3]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack import Document
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

/usr/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 <font size="6">Exercise 3 Intitionaliazing document store containing BPMN-text pairs</font>


In [ ]:

document_store = InMemoryDocumentStore()
doc_embedder = SentenceTransformersDocumentEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)
doc_embedder.warm_up()

file_paths = [
    "Exces_bagagePair.txt",
    "Passenger_ArrivalPair.txt",
    "Passenger_Securitypair.txt",
    "Delayed_BaggagePair.txt"
]

docs = []
for file_path in file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read().strip()
       
        doc = Document(content=content, meta={"source": file_path})
        docs.append(doc)
       
embedded_docs = doc_embedder.run(docs)

document_store.write_documents(embedded_docs["documents"])

print(f"✅ Successfully loaded and embedded {len(docs)} documents into the document store.")


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.98it/s]

✅ Successfully loaded and embedded 4 documents into the document store.


In [6]:
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.embedders import SentenceTransformersTextEmbedder

retriever = InMemoryEmbeddingRetriever(document_store=document_store)
text_embedder = SentenceTransformersTextEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)


<font size="6">Exercise 3(1) ChatPromptBuilder that receives a query containing a BPMN model and an example BPMN-text pair</font>


In [ ]:
from haystack.components.builders import ChatPromptBuilder

Third_prompt_template = ChatPromptBuilder(
    template=[
        ChatMessage.from_user(
            """
You are an expert in BPMN. Your task is to generate a fluent natural-language description of a given BPMN model and annotate every process element using BPMN tags.

First, study this example that shows a BPMN model and its corresponding plain-language description:

{{example_pair}}

Now, generate a new annotated description for the following BPMN model:

{{new_bpmn}}

Instructions:
- Write a clear, step-by-step description of the process.
- Annotate every element using the correct tag:
  • Task → <bpmn:task>Exact Task Name</bpmn:task>
  • Start Event → <bpmn:startEvent>Exact Event Text</bpmn:startEvent>
  • End Event → <bpmn:endEvent>Exact Event Text</bpmn:endEvent>
  • Exclusive Gateway → <bpmn:exclusiveGateway>Exact Gateway Question</bpmn:exclusiveGateway>
- Use ONLY the exact names that appear in the BPMN model.
- Do NOT add any extra text, explanations, or formatting—only return the annotated description.

Generated Annotated Description:

"""
        )
    ],
    required_variables=["example_pair","new_bpmn"]  
)

 <font size="6">Exercise 3 A matching generator</font>


In [ ]:
from haystack_integrations.components.generators.ollama import  OllamaChatGenerator
third_chat_generator = OllamaChatGenerator(
    model="llama3.1:8b",
    url="http://localhost:11434",
    timeout=30 * 60,
    generation_kwargs={
        "num_ctx": 4096,
        "temperature": 0.7, 
    }
)

In [ ]:
from haystack import Pipeline

third_pipeline = Pipeline()
third_pipeline.add_component("text_embedder", text_embedder)
third_pipeline.add_component("retriever", retriever)
third_pipeline.add_component("prompt_builder", Third_prompt_template)
third_pipeline.add_component("llm", third_chat_generator)

third_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
third_pipeline.connect("retriever.documents", "prompt_builder.example_pair")
third_pipeline.connect("prompt_builder.prompt", "llm.messages")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: OllamaChatGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (list[float])
  - retriever.documents -> prompt_builder.example_pair (list[Document])
  - prompt_builder.prompt -> llm.messages (list[ChatMessage])

In [ ]:
NEW_BPMN_FILE = "passenger_check_in_machine.bpmn"  # 
with open(NEW_BPMN_FILE, "r", encoding="utf-8") as f:
    new_bpmn_content = f.read().strip()

result = third_pipeline.run({
    "text_embedder": {"text": new_bpmn_content},
    "prompt_builder": {"new_bpmn": new_bpmn_content}
})
annotated = result["llm"]["replies"][0].text.strip()
print("\n✅ Generated Annotated Description:\n")
print(annotated)

Batches: 100%|██████████| 1/1 [00:00<00:00, 28.58it/s]



✅ Generated Annotated Description:

Here is the generated annotated description:

The process begins when <bpmn:startEvent>Passenger Arrives at Destination</bpmn:startEvent>. The next step is to have the <bpmn:task>Passenger enters terminal</bpmn:task>, which has an outgoing flow to the <bpmn:task>Passenger goes to the mashine</bpmn:task>. This task's output flows into the <bpmn:task>Passenger gets his luggage</bpmn:task> (not present in this BPMN model, but added for context).

After getting the luggage, the passenger needs to decide whether they want to check-in online or not. The decision is made at the <bpmn:exclusiveGateway>Do you want to check-in online?</bpmn:exclusiveGateway>. If they choose to check-in online, the flow goes to the <bpmn:task>Passenger checks in online</bpmn:task>, and if not, it flows into the <bpmn:task>Passenger checks in at the counter</bpmn:task>.

If the passenger chooses to check-in at the counter, they need to go through security, which is represented 

Passenger_check_in_machine.bpmn
The model were able to annotate  0 events out of 2 ,  4 tasks out of 7  and  0 gateways out of 2
Precision = 0.4 Recall = 0.36 

<font size="6">Exercise 3 Running this pipeline for 2 different BPMN models.</font>


In [ ]:
NEW_BPMN_FILE_2 = "passenger_check_in_counter.bpmn"  # 
with open(NEW_BPMN_FILE_2, "r", encoding="utf-8") as f:
    new_bpmn_content = f.read().strip()

result = third_pipeline.run({
    "text_embedder": {"text": new_bpmn_content},
    "prompt_builder": {"new_bpmn": new_bpmn_content}
})

annotated = result["llm"]["replies"][0].text.strip()
print("\n✅ Generated Annotated Description:\n")
print(annotated)

Batches: 100%|██████████| 1/1 [00:00<00:00, 37.39it/s]



✅ Generated Annotated Description:

The process begins with the <bpmn:startEvent>Start Event</bpmn:startEvent>. The next step is <bpmn:task>Passenger enters terminal</bpmn:task>, where the passenger arrives at the terminal. Following this, the passenger proceeds to <bpmn:task>Passenger goes to the counter</bpmn:task>. The process then leads to <bpmn:task>Passenger has to identify</bpmn:task>, where the passenger identifies themselves. After that, the passenger attends <bpmn:task>Exact Task Name (Activity_0y72hjp)</bpmn:task>.

The process then reaches an exclusive gateway at <bpmn:exclusiveGateway>Does the passenger have luggage?</bpmn:exclusiveGateway>. If the answer is yes, the process leads to <bpmn:task>Luggage Handling</bpmn:task>, which is not present in this model but would be if it were included. However, according to the model, if the answer is no, or we assume that is what "Yes" means here as there's only two options and they are not specified directly (it could also mean no

Passenger_check_in_counter 
The model were able to annotate 2 events out of 2 ,   6 tasks out of 7 and 1 gateways out of 2 
Precision = 0.9  Recall =0.81 

In [4]:
document_store = InMemoryDocumentStore()
doc_embedder = SentenceTransformersDocumentEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)
doc_embedder.warm_up()


<font size="6">Exercise 3 Replacing the BPMN-text pairs in the document store with BPMN-annotated-text pairs already containing tags</font>


In [ ]:

file_paths = [
    "Passenger ArrivalAnotate.txt",
    "Passenger SecurityAnotate.txt",
    "Excess BaggageAnotate.txt",
    "Delayed BaggageAnotate.txt"
]

docs = []
for file_path in file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read().strip()
      
        doc = Document(content=content, meta={"source": file_path})
        docs.append(doc)
       
embedded_docs = doc_embedder.run(docs)

document_store.write_documents(embedded_docs["documents"])

print(f"✅ Successfully loaded and embedded {len(docs)} documents into the document store.")


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.22s/it]

✅ Successfully loaded and embedded 4 documents into the document store.


 <font size="6">Exercise 3 rerunning the pipeline</font>


In [ ]:
NEW_BPMN_FILE = "passenger_check_in_machine.bpmn"  # 
with open(NEW_BPMN_FILE, "r", encoding="utf-8") as f:
    new_bpmn_content = f.read().strip()

result = third_pipeline.run({
    "text_embedder": {"text": new_bpmn_content},
    "prompt_builder": {"new_bpmn": new_bpmn_content}
})
annotated = result["llm"]["replies"][0].text.strip()
print("\n✅ Generated Annotated Description:\n")
print(annotated)

Batches: 100%|██████████| 1/1 [00:00<00:00, 27.77it/s]



✅ Generated Annotated Description:

Here is the generated annotated description:

The process begins with <bpmn:startEvent>Passenger Arrives at Terminal</bpmn:startEvent>.

Next, the passenger <bpmn:task>Passenger enters terminal</bpmn:task>. 

Then, the passenger goes to the machine and performs an action in <bpmn:task>Activity_0h3ea40</bpmn:task>. After that, they go through a gateway where they decide whether or not they have a ticket. The gateway is labeled <bpmn:exclusiveGateway>Do you have a ticket?</bpmn:exclusiveGateway>.

If the passenger does have a ticket, then they proceed to select their seat in <bpmn:task>Activity_0y72hjp</bmn:task>. 

However, if they do not have a ticket, they are directed to buy one at <bpmn:task>Activity_074g467</bpmn:task>.

After either of these tasks is completed, the passenger proceeds to retrieve their boarding pass in <bpmn:task>Activity_09u2bux</bmn:task>. 

Finally, they reach their destination and the process ends with <bpmn:endEvent>End of 

Passenger_check_in_machine
The model were able to annotate  0 events out of 2 ,  5 tasks out of 7  and  0 gateways out of 2
Precision =0.62  Recall = 0.45  

In [ ]:
NEW_BPMN_FILE_2 = "passenger_check_in_counter.bpmn"  # 
with open(NEW_BPMN_FILE_2, "r", encoding="utf-8") as f:
    new_bpmn_content = f.read().strip()

result = third_pipeline.run({
    "text_embedder": {"text": new_bpmn_content},
    "prompt_builder": {"new_bpmn": new_bpmn_content}
})
annotated = result["llm"]["replies"][0].text.strip()
print("\n✅ Generated Annotated Description:\n")
print(annotated)

Batches: 100%|██████████| 1/1 [00:00<00:00, 38.39it/s]



✅ Generated Annotated Description:

Here is the generated annotated description of the BPMN model:

The process begins with a <bpmn:startEvent>Passenger arrives at the terminal</bpmn:startEvent>. 

Next, the passenger enters the terminal in <bpmn:task>Passenger enters terminal</bpmn:task>, which leads to <bpmn:task>Passenger goes to the counter</bpmn:task>. 

At this point, the passenger checks if they have a boarding pass in <bpmn:exclusiveGateway>Do you have a boarding pass?</bpmn:exclusiveGateway>. If yes, they proceed to <bpmn:task>Check-in</bpmn:task>, and then go through security in <bpmn:task>Security Check</bpmn:task>.

However, if the passenger does not have a boarding pass, they need to purchase one at <bpmn:task>Purchase Boarding Pass</bpmn:task>. After purchasing their boarding pass, they proceed to check-in in <bpmn:task>Check-in</bpmn:task>, and then go through security in <bpmn:task>Security Check</bpmn:task>.

After clearing security, the passenger proceeds to the gate

Passenger_check_in_counter 
The model were able to annotate 0 events out of 2,  3 tasks out of 7  and  0 gateways out of 2  
Precision = 0.3  Recall = 0.27